https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset?resource=download

In [1]:
import sys
import nltk
import sklearn
import pandas as pd
import numpy as np

In [2]:
import pickle
from joblib import dump, load

In [3]:
# nltk.download()

In [4]:
print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pd.__version__))
print('NumPy: {}'.format(np.__version__))

Python: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]
NLTK: 3.8.1
Scikit-learn: 1.4.2
Pandas: 2.2.2
NumPy: 1.26.4


In [5]:
df = pd.read_csv('labeled_data.csv')

In [6]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [7]:
df[df['class'] == 0]

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
85,85,3,2,1,0,0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1..."
89,90,3,3,0,0,0,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ..."
110,111,3,3,0,0,0,"""@DevilGrimz: @VigxRArts you're fucking gay, b..."
184,186,3,3,0,0,0,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL..."
202,204,3,2,1,0,0,"""@NoChillPaz: ""At least I'm not a nigger"" http..."
...,...,...,...,...,...,...,...
24576,25084,3,2,1,0,0,this guy is the biggest faggot omfg
24685,25197,3,2,1,0,0,which one of these names is more offensive kik...
24751,25264,3,2,1,0,0,you a pussy ass nigga and I know it nigga.
24776,25289,3,3,0,0,0,you're all niggers


In [8]:
df.columns

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet'],
      dtype='object')

In [9]:
df.drop(columns = ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'] ,inplace = True)

In [10]:
df.duplicated().sum()

0

In [11]:
df.isnull().sum()

class    0
tweet    0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   24783 non-null  int64 
 1   tweet   24783 non-null  object
dtypes: int64(1), object(1)
memory usage: 387.4+ KB


In [13]:
df['class'].unique()

array([2, 1, 0], dtype=int64)

<strong>class:</strong> class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [14]:
text = df["tweet"]
text[:10]

label = df['class']

In [15]:
# Eliminar '!!!' al inicio. Eliminar 'RT'. Sustituir nombres de usuario
# text = text.str.replace(r'^!+', '', regex=True).str.replace(r'^RT\s+', '', regex=True).str.replace(r'@\w+', 'users_name', regex=True)

In [16]:
# Eliminar '!!!' al inicio. Eliminar 'RT' correctamente. Sustituir nombres de usuario. Eliminar espacios en blanco sobrantes
# processed = text.str.replace(r'^!+', '', regex=True).str.replace(r'^RT\s+', '', regex=True).str.replace(r'@\w+', 'users_name', regex=True).str.strip()                                         

In [17]:
# Eliminar '!!!' al inicio. Eliminar 'RT' correctamente en cualquier parte del texto. Sustituir nombres de usuario. Eliminar espacios en blanco sobrantes
processed = text .str.replace(r'^!+', '', regex=True).str.replace(r'\bRT\b\s+', '', regex=True).str.replace(r'@\w+', 'name', regex=True).str.strip()                                                                                   

In [18]:
# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ', regex=True)

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

In [19]:
processed = processed.str.lower()
processed

0        name  as a woman you shouldn t complain about ...
1        name  boy dats cold   tyga dwn bad for cuffin ...
2        name dawg     name  you ever fuck a bitch and ...
3                        name  name she look like a tranny
4        name  the shit you hear about me might be true...
                               ...                        
24778    you s a muthaf   in lie   8220 name  name name...
24779    you ve gone and broke the wrong heart baby  an...
24780    young buck wanna eat     dat nigguh like i ain...
24781                youu got wild bitches tellin you lies
24782      ruffled   ntac eileen dahlia   beautiful col...
Name: tweet, Length: 24783, dtype: object

In [20]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [21]:
#probamos de quitar el stemming

ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [22]:
processed

0        name woman complain clean hous amp man alway t...
1        name boy dat cold tyga dwn bad cuffin dat hoe ...
2        name dawg name ever fuck bitch start cri confu...
3                               name name look like tranni
4        name shit hear might true might faker bitch to...
                               ...                        
24778    muthaf lie 8220 name name name right tl trash ...
24779      gone broke wrong heart babi drove redneck crazi
24780    young buck wanna eat dat nigguh like aint fuck...
24781                       youu got wild bitch tellin lie
24782    ruffl ntac eileen dahlia beauti color combin p...
Name: tweet, Length: 24783, dtype: object

In [23]:
from nltk.tokenize import word_tokenize

all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)

all_words = nltk.FreqDist(all_words)

# Print the result
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 19437
Most common words: [('name', 19475), ('bitch', 11480), ('hoe', 4352), ('128514', 3241), ('http', 3127), ('co', 3013), ('like', 2873), ('fuck', 2269), ('pussi', 2267), ('nigga', 2019), ('get', 1782), ('8220', 1726), ('8221', 1682), ('got', 1612), ('ass', 1598)]


<div class="alert alert-block alert-info">
<b>Tip:</b>Puede ser interesante incluir más palabras en el word features ya que contamos de más registros que en el ejercicio de spam. Puede que nos estemos dejando demasiadas palabras interesantes fuerea de la etiquetación y los modelos pordrían perder precisión por ello.
</div>

<div class="alert alert-block alert-success">
<b>Up to you:</b> Sólo estábamos teniendo en cuenta 1500 palabras de 23839.

En cambio en el clasificador de spam estamos cogiendo 1500 de 8921
</div>

In [24]:
# use the 1500 most common words as features
# word_features = [x[0] for x in all_words.most_common(1500)]

word_features = [x[0] for x in all_words.most_common(6000)] # Lo suyo es apostar por proporciones similaes al ejemplo del spam.


In [25]:
# word_features

Guardamos nuestras word_features en un fichero pkl.

In [26]:
with open('word_features.pkl', 'wb') as f:
    pickle.dump(word_features, f)

In [27]:
# word_features

In [28]:
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

In [29]:
features = find_features(processed[0])
for key, value in features.items():
    if value == True:
        print(key)

name
trash
amp
man
take
alway
hous
woman
clean
complain


In [30]:
# features

In [31]:
# list(features.items())[:10]

In [32]:
messages = list(zip(processed, label))

np.random.seed(1)
np.random.shuffle(messages)

# Call find_features function for each SMS message
feature_set = [(find_features(text), label) for (text, label) in messages]

In [33]:
# feature_set[0]

In [34]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(feature_set, test_size=0.25, random_state=1)

In [35]:
# training

In [36]:
# test

In [37]:
print(len(training))
print(len(test))

18587
6196


## Scikit-learn Classifier with NLTK
Now, we build the training and test set, we can build machine learning model in scikit-learn. We are using the following alogithms and see the performance of each ones,

- KNearestNeighbors

- Random Forest
- Decision Tree

- Naive Bayes

In [38]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

names = ['K Nearest Neighbors', 'Decision Tree', 'Random Forest', 'Naive Bayes']

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MultinomialNB()
]

models = zip(names, classifiers)



In [39]:
models

In [40]:
for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, test)
    print("{} model Accuracy: {}".format(name, accuracy))

K Nearest Neighbors model Accuracy: 0.856036152356359
Decision Tree model Accuracy: 0.881697869593286
Random Forest model Accuracy: 0.9025177533892834
Naive Bayes model Accuracy: 0.8858941252420917


In [41]:
from sklearn.ensemble import VotingClassifier

# Since VotingClassifier can accept list type of models
models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators=models, voting='hard', n_jobs=-1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_ensemble, test)
print("Voting Classifier model Accuracy: {}".format(accuracy))

Voting Classifier model Accuracy: 0.8957391865719819


In [51]:
# text_features

In [42]:
text_features, labels = zip(*test)
prediction = nltk_ensemble.classify_many(text_features)

In [43]:
print(classification_report(labels, prediction))

              precision    recall  f1-score   support

           0       0.43      0.28      0.34       350
           1       0.92      0.95      0.94      4847
           2       0.85      0.84      0.85       999

    accuracy                           0.90      6196
   macro avg       0.74      0.69      0.71      6196
weighted avg       0.89      0.90      0.89      6196



In [44]:
pd.DataFrame( confusion_matrix(labels, prediction),
             index=[['actual', 'actual', 'actual'], ['neither', 'ofensive language', 'hate speech']],
             columns = [['predicted', 'predicted', 'predicted'], ['neither', 'ofensive language', 'hate speech']])

predicted                              
                           neither ofensive language hate speech
actual neither                  98               219          33
       ofensive language       121              4616         110
       hate speech               7               156         836

In [45]:
model_paths = {
    "k_nearest_neighbors": "knn_model.pkl",
    "decision_tree": "decision_tree_model.pkl",
    "random_forest": "random_forest_model.pkl",
    "naive_bayes": "naive_bayes_model.pkl",
    "voting_classifier": "voting_classifier.pkl"
}

In [46]:
def save_models(models, voting_classifier):
    for name, model in models:
        with open(model_paths[name.lower().replace(" ", "_")], 'wb') as f:
            pickle.dump(model, f)

    with open(model_paths["voting_classifier"], 'wb') as f:
        pickle.dump(voting_classifier, f)

In [47]:
models_to_save = list(zip(names, classifiers))

In [48]:
save_models(models_to_save, nltk_ensemble)

In [49]:
# Funcion para cargar los modelos:
model_paths = {
    "k_nearest_neighbors": "knn_model.pkl",
    "decision_tree": "decision_tree_model.pkl",
    "random_forest": "random_forest_model.pkl",
    "naive_bayes": "naive_bayes_model.pkl",
    "voting_classifier": "voting_classifier.pkl"
}

def load_models():
    loaded_models = {}
    for name, path in model_paths.items():
        with open(path, 'rb') as f:
            loaded_models[name] = pickle.load(f)
    return loaded_models

